In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [7]:
from ase.db import connect
from Core import Nanoparticle as NP
from Core import Adsorption as ADS
from MCMC import MonteCarlo
from Core import EnergyCalculator as EC
from Core.LocalEnvironmentFeatureClassifier import TopologicalEnvironmentClassifier
from Core.LocalEnvironmentCalculator import LocalEnvironmentCalculator
import numpy as np

db = connect('/home/riccardo/NiAu_database.db')

In [8]:
def get_TFC(top):
    feature_vector = np.zeros(17)
    i = 0
    for f,n in zip([2, 10, 11, 13, 16],[0,24,24,44,48]):
        feature_vector[f] = abs(n-top[i])
        i += 1
    return feature_vector

In [9]:
def get_training_set(db):
    from Core import GlobalFeatureClassifier as GFC
    training_set = []
    for row in db.select():
        p = NP.Nanoparticle()
        p.add_atoms(row.toatoms())
        #GFC.TopologicalFeatureClassifier(p.get_all_symbols()).compute_feature_vector(p)
        p.set_feature_vector('TFC', get_TFC(row.data.TOP))
        p.set_energy('DFT', row.dft_energy)
        training_set.append(p)
    return training_set
training_set = get_training_set(db)

In [10]:
ordering_energy_calculator = EC.BayesianRRCalculator('TFC')
ordering_energy_calculator.fit(training_set, 'DFT')
coefficents, total_energies = EC.compute_coefficients_for_linear_topological_model(
ordering_energy_calculator.get_coefficients(), ['Au', 'Ni'], 140)
ordering_energy_calculator.set_coefficients(coefficents)
ordering_energy_calculator.set_feature_key('TEC')

Coef symbol_a: Au


In [11]:
ordering_energy_calculator.energy_key = 'BRRR'

In [33]:
Au = 0
Ni = 0
Au2 = -0.28120172 
Au2Ni = -1.00039431 
Au3 = -10.35726243
AuNi = -1.01410455
AuNi2 = -2.08995281
Ni2 = -2.23349599
Ni3 = -2.40469383

In [34]:
from MCMC.MonteCarlo import run_monte_carlo_ordering_adsorbates

start_particle = NP.Nanoparticle()
start_particle.truncated_octahedron(6,1,{'Au':84,'Ni':56})

beta = 200
max_steps = 5000

#ordering_energy_calculator = EC.BayesianRRCalculator('TEC')
#ordering_energy_calculator.set_coefficients(coefficents)
adsorbates_energy_calculator = EC.BayesianRRCalculator('ADS')
sergey_coefficients = [Au, Ni, Au2, AuNi, Ni2, Au3, Au2Ni, AuNi2, Ni3]
adsorbates_energy_calculator.set_coefficients(sergey_coefficients)
n_adsorbates = 10
local_feature_classifier = TopologicalEnvironmentClassifier(LocalEnvironmentCalculator(), ['Au', 'Ni'])

[best_particle, accepted_energies] = run_monte_carlo_ordering_adsorbates(beta, max_steps, start_particle, ordering_energy_calculator, adsorbates_energy_calculator, n_adsorbates, local_feature_classifier)

Step: 2000
Lowest energy: -315.11615950898846
Step: 4000
Lowest energy: -322.5805533166482
Step: 6000
Lowest energy: -322.6131924506158
Step: 8000
Lowest energy: -322.93011765815845
Step: 10000
Lowest energy: -322.98365490873226
Step: 12000
Lowest energy: -322.98365490873226
Step: 14000
Lowest energy: -322.98365490873226


In [35]:
from ase.visualize import view
from Core.Adsorption import PlaceAddAtoms

best_p = NP.Nanoparticle()
best_p.build_from_dictionary(best_particle)
best_p.construct_neighbor_list()
best_p.construct_adsorption_list()
ads = PlaceAddAtoms(best_p.get_all_symbols())
ads.bind_particle(best_p)
for f in best_particle['ads']:
    try:
        site = list(best_p.get_site_atom_indices(f))
        ads.place_add_atom(best_p, 'O', [site])
    except:
        print(f)


view(best_p.get_ase_atoms())

In [152]:
best_p.write('porcodio.xyz')

In [ ]:
start_particle = NP.Nanoparticle()
start_particle.truncated_octahedron(6,1,{'Au':84,'Ni':56})

beta = 200
max_steps = 1000

adsorbate = None
local_feature_classifier = TopologicalEnvironmentClassifier(LocalEnvironmentCalculator()
                                                            , ['Au', 'Ni'])

[best_particle, accepted_energies] = MonteCarlo.run_monte_carlo(
                                            beta, max_steps, start_particle, energy_calculator,
                                            local_feature_classifier, adsorbate
                                                            )

In [76]:
np.random.randint(2)

1

In [74]:
best_p.swap_symbols(list(np.random.choice(range(10),0)))

In [61]:
from MCMC import MonteCarlo as MC
from Core.Nanoparticle import Nanoparticle
from Core.GlobalFeatureClassifier import AdsorptionFeatureVector
from Core.EnergyCalculator import BayesianRRCalculator

best_p = Nanoparticle()
best_p.build_from_dictionary(best_particle)
best_p.construct_neighbor_list()
best_p.construct_adsorption_list()


beta = 200
max_steps = 5000
energy_calculator = BayesianRRCalculator('ADS')
sergey_coefficients = [Au, Ni, Au2, AuNi, Ni2, Au3, Au2Ni, AuNi2, Ni3]
energy_calculator.set_coefficients(sergey_coefficients)
n_adsorbates = 20

[best_particle, accepted_energies] = MC.run_monte_carlo_for_adsorbates(beta, max_steps, best_p, energy_calculator, n_adsorbates)

Step: 2000
Lowest energy: -47.92267876
Step: 4000
Lowest energy: -48.093876599999994
Step: 6000
Lowest energy: -48.093876599999994


In [63]:
from ase.visualize import view
from Core.Adsorption import PlaceAddAtoms

best_p = Nanoparticle()
best_p.build_from_dictionary(best_particle)
best_p.construct_neighbor_list()
best_p.construct_adsorption_list()
ads = PlaceAddAtoms(best_p.get_all_symbols())
ads.bind_particle(best_p)
for f in best_particle['ads']:
    site = list(best_p.get_site_atom_indices(f))
    ads.place_add_atom(best_p, 'O', [site])


view(best_p.get_ase_atoms())

<Popen: returncode: None args: ['/home/riccardo/anaconda3/bin/python3.9', '-...>

In [ ]:
best_p = Nanoparticle()
best_p.build_from_dictionary(best_particle)
best_p.construct_neighbor_list()
best_p.construct_adsorption_list()
AdsorptionFeatureVector(best_p.get_all_symbols()).compute_feature_vector(best_p)
best_p.get_feature_vector('ADS')

In [ ]:
AdsorptionFeatureVector(best_p.get_all_symbols()).features_type

In [115]:
from MCMC.RandomExchangeOperator import RandomExchangeOperator

exchange_operator = RandomExchangeOperator(0.5)
exchange_operator.bind_adsorbates(best_p,5)


([(41, 58)], [])

In [131]:
exchange_operator.coupled_random_exchange(best_p)

([], [(541, 485)])

In [132]:
best_p.get_indices_of_adsorbates()

array([142, 199, 286, 393, 541])

In [133]:
best_p.get_indices_of_adsorbates()

array([142, 199, 286, 393, 541])